In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import logging
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define file paths
DATASET_PATH = "/Users/akashthanneeru/Desktop/INADS_Data/Data/Indexed_Dataset_Timestamp_Processed.csv"
MODEL_SAVE_PATH = "/Users/akashthanneeru/Desktop/INADS_Data/Models/Device_Layer_AE_GRU.keras"

# Load dataset
logging.info("Loading dataset for Device Layer...")
df = pd.read_csv(DATASET_PATH)

# Define selected features for Device Layer
device_features = [
    "Dst Port", "Fwd Pkts/s", "Bwd Pkts/s",
    "Fwd Pkt Len Max", "Bwd Pkt Len Min",
    "Init Fwd Win Byts", "Init Bwd Win Byts",
    "Active Max", "Active Mean", "Active Min", "Idle Max",
    "Hour", "Elapsed_Time"
]

# Encode labels
logging.info("Encoding labels...")
label_encoder = LabelEncoder()
df["Label"] = label_encoder.fit_transform(df["Label"])

# Split data into features (X) and target (y)
X = df[device_features]
y = df["Label"]

# Normalize the feature values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Autoencoder model for feature extraction
logging.info("Building Autoencoder for Device Layer...")
input_dim = X_train.shape[1]

input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation="relu")(input_layer)
encoded = Dense(32, activation="relu")(encoded)
decoded = Dense(64, activation="relu")(encoded)
decoded = Dense(input_dim, activation="linear")(decoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer="adam", loss="mse")

# Train Autoencoder
logging.info("Training Autoencoder...")
autoencoder.fit(X_train, X_train, epochs=20, batch_size=128, validation_data=(X_test, X_test), verbose=1)

# Extract latent features
encoder = Model(inputs=input_layer, outputs=encoded)
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

# Reshape for GRU input
X_train_encoded = X_train_encoded.reshape((X_train_encoded.shape[0], 1, X_train_encoded.shape[1]))
X_test_encoded = X_test_encoded.reshape((X_test_encoded.shape[0], 1, X_test_encoded.shape[1]))

# Define GRU model for final classification
logging.info("Building GRU model for Device Layer...")
gru_model = Sequential([
    GRU(32, return_sequences=False, input_shape=(1, X_train_encoded.shape[2])),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(len(label_encoder.classes_), activation="softmax")  # Multi-class classification
])

# Compile GRU model
gru_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train GRU model
logging.info("Training Device Layer GRU model...")
start_time = time.time()
gru_model.fit(X_train_encoded, y_train, epochs=30, batch_size=64, validation_data=(X_test_encoded, y_test), verbose=1)
training_time = time.time() - start_time
logging.info(f"GRU model training completed in {training_time:.2f} seconds.")

# Evaluate model
logging.info("Evaluating Device Layer GRU model...")
y_pred = np.argmax(gru_model.predict(X_test_encoded), axis=1)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
logging.info("\n" + report)

# Save the model in the recommended format
gru_model.save(MODEL_SAVE_PATH)
logging.info(f"Model saved successfully at: {MODEL_SAVE_PATH}")

2025-03-17 21:33:41,006 - INFO - Loading dataset for Device Layer...
2025-03-17 21:33:51,385 - INFO - Encoding labels...
2025-03-17 21:33:52,042 - INFO - Building Autoencoder for Device Layer...
2025-03-17 21:33:52,224 - INFO - Training Autoencoder...


Epoch 1/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 491us/step - loss: 0.0265 - val_loss: 0.0012
Epoch 2/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 491us/step - loss: 0.0017 - val_loss: 2.6630e-04
Epoch 3/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 497us/step - loss: 9.3465e-04 - val_loss: 8.7764e-04
Epoch 4/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 473us/step - loss: 8.4828e-04 - val_loss: 2.6690e-04
Epoch 5/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 491us/step - loss: 6.2839e-04 - val_loss: 0.0014
Epoch 6/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 476us/step - loss: 3.7729e-04 - val_loss: 7.6398e-05
Epoch 7/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 500us/step - loss: 3.7583e-04 - val_loss: 9.4411e-05
Epoch 8/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 475us/step - loss: 3.4056e-04 - val_loss: 0.0013
Epoch 9/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 491us/step - loss: 3.9021e-04 - val_loss: 1.8247e-04
Epoch 10/20
14190/14190 ━━━━━━━━━━━━━━━━━━━━ 7s 478us/step - loss: 2.7956e-04 - val_loss: 1.2144e-04
Epoch 11/20
14190/1

2025-03-17 21:36:33,290 - INFO - Building GRU model for Device Layer...
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-03-17 21:36:33,321 - INFO - Training Device Layer GRU model...


Epoch 1/30
28379/28379 ━━━━━━━━━━━━━━━━━━━━ 22s 735us/step - accuracy: 0.9630 - loss: 0.1097 - val_accuracy: 0.9697 - val_loss: 0.0705
Epoch 2/30
28379/28379 ━━━━━━━━━━━━━━━━━━━━ 21s 736us/step - accuracy: 0.9697 - loss: 0.0724 - val_accuracy: 0.9698 - val_loss: 0.0714
Epoch 3/30
28379/28379 ━━━━━━━━━━━━━━━━━━━━ 22s 760us/step - accuracy: 0.9699 - loss: 0.0708 - val_accuracy: 0.9698 - val_loss: 0.0687
Epoch 4/30
28379/28379 ━━━━━━━━━━━━━━━━━━━━ 22s 761us/step - accuracy: 0.9696 - loss: 0.0707 - val_accuracy: 0.9698 - val_loss: 0.0691
Epoch 5/30
28379/28379 ━━━━━━━━━━━━━━━━━━━━ 22s 774us/step - accuracy: 0.9699 - loss: 0.0698 - val_accuracy: 0.9698 - val_loss: 0.0682
Epoch 6/30
28379/28379 ━━━━━━━━━━━━━━━━━━━━ 22s 767us/step - accuracy: 0.9697 - loss: 0.0700 - val_accuracy: 0.9697 - val_loss: 0.0690
Epoch 7/30
28379/28379 ━━━━━━━━━━━━━━━━━━━━ 22s 769us/step - accuracy: 0.9696 - loss: 0.0697 - val_accuracy: 0.9698 - val_loss: 0.0680
Epoch 8/30
28379/28379 ━━━━━━━━━━━━━━━━━━━━ 22s 790us/s

2025-03-17 21:47:21,026 - INFO - GRU model training completed in 647.70 seconds.
2025-03-17 21:47:21,026 - INFO - Evaluating Device Layer GRU model...


14190/14190 ━━━━━━━━━━━━━━━━━━━━ 3s 213us/step


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2025-03-17 21:47:25,553 - INFO - 
   

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# File Paths
DEVICE_MODEL_PATH = "/Users/akashthanneeru/Desktop/INADS_Data/Models/Device_Layer_AE_GRU.keras"
DATASET_PATH = "/Users/akashthanneeru/Desktop/INADS_Data/Data/Indexed_Dataset_Timestamp_Processed.csv"

# Load Dataset
logging.info("Loading dataset for Device Layer analysis...")
df = pd.read_csv(DATASET_PATH)

# 🔹 Ensure we use the **correct** 32 features (Replace this with actual trained features)
DEVICE_FEATURES = [
    "Dst Port", "Fwd Pkts/s", "Bwd Pkts/s", "Fwd Pkt Len Max", "Bwd Pkt Len Min",
    "Init Fwd Win Byts", "Init Bwd Win Byts", "Active Max", "Active Min",
    "Idle Max", "Hour", "Weekday", "Elapsed_Time",
    "Flow Duration", "Flow Byts/s", "Flow Pkts/s", "Pkt Len Min", "Pkt Len Max",
    "Fwd IAT Mean", "Bwd IAT Mean", "Down/Up Ratio", "Subflow Fwd Pkts",
    "Active Mean", "Idle Mean", "Fwd Seg Size Avg", "Bwd Seg Size Avg",
    "TotLen Fwd Pkts", "TotLen Bwd Pkts", "Flow IAT Mean", "Flow IAT Std",
    "Init Fwd Win Byts", "Init Bwd Win Byts"
]

# Standardize Features
scaler = StandardScaler()
df[DEVICE_FEATURES] = scaler.fit_transform(df[DEVICE_FEATURES])

# Load Trained Device Layer Model
logging.info("Loading Device Layer AE-GRU model...")
device_model = load_model(DEVICE_MODEL_PATH)

# 🔹 Reshape Input to **match the model's expected shape**
X_device = df[DEVICE_FEATURES].values.reshape(-1, 1, 32)  # (samples, time_steps=1, features=32)

# ============== Autoencoder Reconstruction Error Analysis ==================
def get_autoencoder_reconstruction_error(model, X):
    """
    Computes the reconstruction error for an autoencoder model.
    """
    reconstructed = model.predict(X)
    return np.mean(np.abs(X - reconstructed), axis=1)

logging.info("Extracting Autoencoder reconstruction errors...")
ae_reconstruction_errors = get_autoencoder_reconstruction_error(device_model, X_device)

# 🔹 Visualize Reconstruction Errors
plt.figure(figsize=(12, 6))
sns.histplot(ae_reconstruction_errors, bins=50, kde=True, color='red')
plt.title("Reconstruction Error Distribution - Device Layer")
plt.show()

# ============== GRU Activation Analysis ==================
def get_gru_hidden_states(model, X):
    """
    Extracts hidden states from the first GRU layer.
    """
    extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[1].output)  # Extracting GRU layer
    return extractor.predict(X)

logging.info("Extracting GRU hidden states...")
gru_hidden_states = get_gru_hidden_states(device_model, X_device)

# 🔹 Visualize GRU Hidden State Evolution
plt.figure(figsize=(12, 6))
sns.heatmap(gru_hidden_states.mean(axis=0).reshape(1, -1), cmap='coolwarm')
plt.title("GRU Hidden State Evolution - Device Layer")
plt.show()

# ============== Correlation Heatmap ==================
plt.figure(figsize=(12, 6))
sns.heatmap(df[DEVICE_FEATURES].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap - Device Layer Features")
plt.show()

# ============== Time-Series Feature Trends ==================
plt.figure(figsize=(12, 6))
for feature in DEVICE_FEATURES[:5]:  # Visualizing first 5 features
    plt.plot(df[feature][:200], label=feature)
plt.legend()
plt.title("Time-Series Trends - First 5 Features (Device Layer)")
plt.show()

logging.info("Device Layer analysis complete.")

2025-03-17 22:48:24,574 - INFO - Loading dataset for Device Layer analysis...
2025-03-17 22:48:38,356 - INFO - Loading Device Layer AE-GRU model...
2025-03-17 22:48:38,846 - INFO - Extracting Autoencoder reconstruction errors...


70948/70948 ━━━━━━━━━━━━━━━━━━━━ 16s 223us/step


ValueError: operands could not be broadcast together with shapes (2270317,1,32) (2270317,7) 